<a href="https://colab.research.google.com/github/arjunparmar/TUNEX/blob/main/Deepanshu/Testing/test_Web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import imutils
import h5py

In [ ]:
cascade = cv2.CascadeClassifier('/content/drive/MyDrive/haarcascade_frontalface_default.xml')
model = load_model("/content/drive/MyDrive/model_Four.h5")

In [ ]:
classes = ['Afraid', 'Angry', 'Disgust', 'Happy', 'Neutral', 'Sad', 'Suprised']

In [ ]:
!pip install ffmpeg-python


from IPython.display import HTML, Javascript, display
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
import ffmpeg

video_file_test = '/content/drive/MyDrive/Video/osy_test.mp4' 
  

VIDEO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var my_btn_txt = document.createTextNode("Press to start recording");
my_btn.appendChild(my_btn_txt);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);
var base64data = 0;
var reader;
var recorder, videoStream;
var recordButton = my_btn;
var handleSuccess = function(stream) {
  videoStream = stream;
  var options = {  
    mimeType : 'video/webm;codecs=vp9'  
  };            
  recorder = new MediaRecorder(stream, options);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('video');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);
    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
  };
recordButton.innerText = "Recording... press to stop";
navigator.mediaDevices.getUserMedia({video: true}).then(handleSuccess);
function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      videoStream.getVideoTracks()[0].stop();
      recordButton.innerText = "Saving the recording... Please wait!"
  }
}
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}
var data = new Promise(resolve=>{
recordButton.onclick = ()=>{
toggleRecording()
sleep(2000).then(() => {
  // wait 2000ms for the data to be available
  resolve(base64data.toString())
});
}
});
      
</script>
"""

def start_webcam():
  js = Javascript('''
    async function startWebcam() {
      const div = document.createElement('div');
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();
      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      
      return;
    }
    ''')
  
  display(js)
  data = eval_js('startWebcam()')
  
    
start_webcam()

def get_video():
  display(HTML(VIDEO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  return binary

In [ ]:
videofile = get_video()
with open(video_file_test, 'wb') as f:
  f.write(videofile)

In [ ]:
from google.colab.patches import cv2_imshow

capture = cv2.VideoCapture(video_file_test)

while capture.isOpened():

    check, image = capture.read()
    image = imutils.resize(image, width=400)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face  = cascade.detectMultiScale(gray, 1.3, 8)

    for (x, y, w, h) in face:
        cv2.rectangle(image, (x, y), ((x + w), (y + h)), (110, 240, 4), 2)
        cv2.rectangle(image, (x, y-30), ((x + w), y), (110, 240, 4), -2)
        roi = gray[y:y+h, x:x+w]
        roi = cv2.resize(roi, (48,48), interpolation=cv2.INTER_AREA)
        roi = roi.astype("float")/255.0
        roi = img_to_array(roi)
        roi = tf.expand_dims(roi, axis=0)

        emotion = model.predict(roi)[0]
        label = classes[emotion.argmax()]
        
        cv2.putText(image, label, (x+20, y-10), cv2.FONT_HERSHEY_DUPLEX, 0.6, (255,255,255), 2)
    else:
        cv2.putText(image, 'No Face Detected', (20, 10), cv2.FONT_HERSHEY_DUPLEX, 0.6, (255,255,255), 2)

    cv2_imshow(image)

    if cv2.waitKey(1) in {27, 32, 13}:
        break

capture.release()
cv2.destroyAllWindows()